In [1]:
import pandas as pd
import numpy as np

In [2]:
xls = pd.ExcelFile("tabela_07.A.03_Consumo_cimento_27.xlsx", engine="openpyxl"
)

In [3]:
xls.sheet_names

['2003',
 '2004',
 '2005',
 '2006',
 '2007',
 '2008',
 '2009',
 '2010',
 '2011',
 '2012',
 '2013',
 '2014',
 '2015',
 '2016',
 '2017',
 '2018',
 '2019',
 '2020',
 '2021',
 '2022']

In [4]:
REGIONS = [
    "REGIÃO NORTE",
    "REGIÃO  NORDESTE",
    "REGIÃO SUDESTE",
    "REGIÃO  SUL",
    "CENTRO-OESTE",
    "TOTAL  BRASIL",
]

STATES = [
    "ACRE",
    "ALAGOAS",
    "AMAPÁ",
    "AMAZONAS",
    "BAHIA",
    "CEARÁ",
    #"CENTRO-OESTE",
    "DISTRITO FEDERAL",
    "ESPÍRITO SANTO",
    "GOIÁS",
    "MARANHÃO",
    "MATO GROSSO",
    "MATO GROSSO DO SUL",
    "MINAS GERAIS",
    "PARANÁ",
    "PARAÍBA",
    "PARÁ",
    "PERNAMBUCO",
    "PIAUÍ",
    #"REGIÃO  NORDESTE",
    #"REGIÃO  SUL",
    #"REGIÃO NORTE",
    #"REGIÃO SUDESTE",
    "RIO DE JANEIRO",
    "RIO GRANDE DO NORTE",
    "RIO GRANDE DO SUL",
    "RONDÔNIA",
    "RORAIMA",
    "SANTA CATARINA",
    "SERGIPE",
    "SÃO PAULO",
    "TOCANTINS",
]

STATE_INITIALS = [
    "AC",
    "AL",
    "AM",
    "AP",
    "BA",
    "CE",
    "DF",
    "ES",
    "GO",
    "MA",
    "MG",
    "MS",
    "MT",
    "PA",
    "PB",
    "PE",
    "PI",
    "PR",
    "RJ",
    "RN",
    "RO",
    "RR",
    "RS",
    "SC",
    "SE",
    "SP",
    "TO",
]

MONTHS = ["01", "02", "03", "04", "05", "06", "07", "08", "09", "10", "11", "12"]

In [5]:
def read_tab_and_preprocess(tab):
    # Lê a aba 'tab' do excel e ignore as 4 primeiras linhas onde se encontra os cabeçalhos
    df = pd.read_excel(xls, tab, skiprows=4)
    df = df.rename(
        {
            "LOCALIDADE": "Estado",
            "Unnamed: 0": "Estado",
            "e ESTADOS": "Estado",
            "e REGIÕES": "Estado",
        },
        axis=1,
    )
    df = df.loc[:, ~df.columns.str.contains("^Unnamed")]
    df = df.replace({"...": np.nan})
    df = df.replace({"....": np.nan})
    df = df.replace({"-": np.nan})

    # Remove total por região
    df["STATES_KEEP"] = df["Estado"].apply(lambda x: x not in REGIONS)
    df = df[df["STATES_KEEP"]]

    # Remove outros valores que não são dos estados e.g. valor de ajuste
    df["STATES_KEEP"] = df["Estado"].apply(lambda x: x in STATES)
    df = df[df["STATES_KEEP"]]
    df = df.drop(["STATES_KEEP"], axis=1)
    df = df.drop(["TOTAL"], axis=1)

    # Ordena todo o DF pelo nome do estado
    df = df.sort_values(by="Estado")
    df = df.infer_objects()
    df = df.reset_index(drop=True)

    # Troca nome do estado por suas iniciais
    state_to_initials = dict(zip(STATES, STATE_INITIALS))
    df["Estado"] = df["Estado"].apply(lambda x: state_to_initials[x])
    return df


In [6]:
def transpose_df(df):
    # Cria Dataframe com duas colunas contendo estados e ano-mês
    data = data = [year + month for month in MONTHS]
    estados = df["Estado"].values
    datas = np.array([[d] * len(estados) for d in data]).flatten()
    estados = np.array([estados] * len(data)).flatten()
    df_tmp = pd.DataFrame({"data": datas, "estados": estados})
    df_tmp["data"] = df_tmp["data"].astype("int")
    
    # Extraí os valores de produção do cimento que estão horizontais
    # e os coloca na posição vertocal no novo dataframe
    producao = []
    for i in range(df.shape[0]):
        producao.append(df.drop("Estado", axis=1).T[i].values)
    producao = np.array(producao).reshape(-1, 1).squeeze()
    df_tmp["consumo"] = producao
    producao = np.array(producao).flatten()
    df_tmp["consumo"] = producao
    
    return df_tmp

In [7]:
# Converte todos as abas de anos do excel em dataframes
# Empilha todos os dataframes
# Salva no formato .csv
dataframes = []
for year in xls.sheet_names:
    df = read_tab_and_preprocess(year)
    df = transpose_df(df)
    dataframes.append(df)

df = pd.concat(dataframes)
df = df.reset_index(drop=True)
df.to_csv("consumo_mensal_cimento_2003_2022.csv", index=False)

In [8]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
data,6468.0,201258.262523,575.762580,200301.0,200801.0,201301.0,201801.000,202212.0
consumo,5253.0,154754.148237,206530.897519,1466.0,40132.0,81660.0,190782.485,1491696.0


In [9]:
df

,data,estados,consumo
0,200301,AC,3260.0
1,200301,AL,2191.0
2,200301,AM,2154.0
3,200301,AP,2643.0
4,200301,BA,2546.0
...,...,...,...
6463,202212,RS,NaN
6464,202212,SC,NaN
6465,202212,SE,NaN
6466,202212,SP,NaN


In [10]:
df.isna().sum()

data          0
estados       0
consumo    1215
dtype: int64

In [12]:
df[df["consumo"].isna()]

,data,estados,consumo
3554,201401,AM,NaN
3555,201401,AP,NaN
3556,201401,BA,NaN
3557,201401,CE,NaN
3558,201401,DF,NaN
...,...,...,...
6463,202212,RS,NaN
6464,202212,SC,NaN
6465,202212,SE,NaN
6466,202212,SP,NaN
